#  AT82.05 Artificial Intelligence: Natural Language Understanding (NLU)

## A4: Do You AGREE

### Name: Arya Shah
### StudentID: st125462

-----------

In this assignment, I will explore training a pre-trained model like BERT from scratch, focusing on leveraging text embeddings to capture semantic similarity. Additionally, we will explore how to adapt the loss function for tasks like Natural Language Inference (NLI) to enhance the model’s ability to understand semantic relationships between texts

You can find the GitHub Repository for the assignment here:
- https://github.com/aryashah2k/NLP-NLU (Complete Web App)
- https://github.com/aryashah2k/NLP-NLU/tree/main/notebooks (Assignment Notebooks)
- https://github.com/aryashah2k/NLP-NLU/tree/main/reports (Assignment Reports)

# Task 1: Training BERT from Scratch ✅

Based on Masked Language Model/BERT-update.ipynb, modify as follows: (2 points)
1) Implement Bidirectional Encoder Representations from Transformers (BERT) from scratch, following the concepts learned in class. ✅
2) Train the model on a suitable dataset. Ensure to source this dataset from reputable public databases  or repositories. It is imperative to give proper credit to the dataset source in your documentation.✅
3) Save the trained model weights for later use in Task 2.✅

In [ ]:
### Dataset Source Attribution and Credits

"""
Dataset Documentation

1. BookCorpus Dataset
-------------------
Description:
    A large collection of free novel books written by unpublished authors.
    Contains approximately 74M sentences and 1B words from 11,038 books

Usage:
    from datasets import load_dataset
    
    # Load full dataset
    dataset = load_dataset('bookcorpus')
    
    # Load specific splits
    train_test = load_dataset('bookcorpus', split='train+test')
    
    # Load percentage of data
    partial_data = load_dataset('bookcorpus', split='train[:10%]')[1]

2. SNLI (Stanford Natural Language Inference) Dataset
-------------------------------------------------
Description:
    A collection of 570k human-written English sentence pairs labeled for 
    balanced classification with entailment, contradiction, and neutral labels.

Structure:
    - Total Instances: 570,152
    - Splits:
        - Train: 550,152
        - Validation: 10,000
        - Test: 10,000

Data Fields:
    - premise: str  # Base statement for comparison
    - hypothesis: str  # Statement to be evaluated against premise
    - label: int  # 0: entailment, 1: neutral, 2: contradiction, -1: no consensus

Average Token Counts:
    - Premise: 14.1 tokens
    - Hypothesis: 8.3 tokens

Usage:
    from datasets import load_dataset
    
    dataset = load_dataset('stanfordnlp/snli')
    
    # Filter invalid labels
    valid_data = dataset.filter(lambda x: x['label'] != -1)

Note:
    Each premise appears in only one split, though it may be used in 
    multiple examples with different hypotheses
"""

In [6]:
import math
import numpy as np
import re
import logging
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from datasets import load_dataset
from tqdm.auto import tqdm
import os
import json
from datetime import datetime
from transformers import get_cosine_schedule_with_warmup

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('bert_training.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

def get_free_gpu():
    """Get the GPU with the most available memory."""
    if not torch.cuda.is_available():
        return torch.device('cpu')
    
    # Get the number of GPUs
    n_gpus = torch.cuda.device_count()
    if n_gpus == 0:
        return torch.device('cpu')
    
    # Find GPU with most free memory
    max_free_memory = 0
    selected_gpu = 0
    
    for gpu_id in range(n_gpus):
        try:
            # Get free memory for this GPU
            free_memory = torch.cuda.get_device_properties(gpu_id).total_memory - torch.cuda.memory_allocated(gpu_id)
            if free_memory > max_free_memory:
                max_free_memory = free_memory
                selected_gpu = gpu_id
        except:
            continue
    
    device = torch.device(f'cuda:{selected_gpu}')
    logger.info(f"Selected GPU {selected_gpu} with {max_free_memory/1024/1024:.2f}MB free memory")
    return device

# Set device and seeds for reproducibility
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
device = get_free_gpu()
logger.info(f"Using device: {device}")

# Model configuration
class BertConfig:
    def __init__(self):
        # Model architecture
        self.vocab_size = None  # Will be set after data loading
        self.hidden_size = 256
        self.num_hidden_layers = 6
        self.num_attention_heads = 8
        self.intermediate_size = 1024
        
        # Dropout and normalization
        self.hidden_dropout_prob = 0.1
        self.attention_probs_dropout_prob = 0.1
        self.layer_norm_eps = 1e-12
        
        # Sequence parameters
        self.max_position_embeddings = 128
        self.max_len = 128
        self.type_vocab_size = 2
        self.pad_token_id = 0
        
        # Special tokens
        self.mask_token_id = 3
        self.cls_token_id = 1
        self.sep_token_id = 2
        self.pad_token_id = 0
        
        # Training parameters
        self.learning_rate = 1e-4
        self.batch_size = 64
        self.gradient_accumulation_steps = 4
        self.weight_decay = 0.01
        self.adam_epsilon = 1e-8
        self.warmup_ratio = 0.1

class BertLayerNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-12):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.bias = nn.Parameter(torch.zeros(hidden_size))
        self.variance_epsilon = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        variance = (x - mean).pow(2).mean(-1, keepdim=True)
        x = (x - mean) / torch.sqrt(variance + self.variance_epsilon)
        return self.weight * x + self.bias

class BertEmbeddings(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size)
        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        
    def forward(self, input_ids, token_type_ids=None, position_ids=None):
        seq_length = input_ids.size(1)
        if position_ids is None:
            position_ids = torch.arange(seq_length, dtype=torch.long, device=input_ids.device)
            position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)
            
        words_embeddings = self.word_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)
        
        embeddings = words_embeddings + position_embeddings + token_type_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

class BertSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = config.hidden_size // config.num_attention_heads
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        # Initialize with smaller values for stability
        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)
        
        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)
        self.layer_norm = BertLayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        
        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        for module in [self.query, self.key, self.value]:
            module.weight.data.normal_(mean=0.0, std=0.02)
            if module.bias is not None:
                module.bias.data.zero_()

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)
    
    def forward(self, hidden_states, attention_mask=None):
        query_layer = self.transpose_for_scores(self.query(hidden_states))
        key_layer = self.transpose_for_scores(self.key(hidden_states))
        value_layer = self.transpose_for_scores(self.value(hidden_states))
        
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        
        if attention_mask is not None:
            attention_scores = attention_scores + attention_mask
            
        attention_probs = nn.Softmax(dim=-1)(attention_scores)
        attention_probs = self.dropout(attention_probs)
        
        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)
        
        return context_layer

class BertLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.attention = BertSelfAttention(config)
        self.intermediate = nn.Linear(config.hidden_size, config.intermediate_size)
        self.output = nn.Linear(config.intermediate_size, config.hidden_size)
        self.LayerNorm1 = BertLayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.LayerNorm2 = BertLayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.activation = F.gelu
        
    def forward(self, hidden_states, attention_mask=None):
        attention_output = self.attention(hidden_states, attention_mask)
        attention_output = self.dropout(attention_output)
        attention_output = self.LayerNorm1(attention_output + hidden_states)
        
        intermediate_output = self.intermediate(attention_output)
        intermediate_output = self.activation(intermediate_output)
        
        layer_output = self.output(intermediate_output)
        layer_output = self.dropout(layer_output)
        layer_output = self.LayerNorm2(layer_output + attention_output)
        
        return layer_output

class BertModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.embeddings = BertEmbeddings(config)
        self.encoder = nn.ModuleList([BertLayer(config) for _ in range(config.num_hidden_layers)])
        self.pooler = nn.Linear(config.hidden_size, config.hidden_size)
        self.pooler_activation = nn.Tanh()
        
        # MLM head
        self.mlm_head = nn.Linear(config.hidden_size, config.vocab_size)
        
        # Enable gradient checkpointing for memory efficiency
        self.gradient_checkpointing = False
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, masked_lm_labels=None):
        if attention_mask is None:
            attention_mask = torch.ones_like(input_ids)
        
        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
        extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype)
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
        
        embedding_output = self.embeddings(input_ids, token_type_ids)
        
        hidden_states = embedding_output
        
        for layer in self.encoder:
            if self.gradient_checkpointing and self.training:
                def create_custom_forward(module):
                    def custom_forward(*inputs):
                        return module(*inputs)
                    return custom_forward
                
                hidden_states = torch.utils.checkpoint.checkpoint(
                    create_custom_forward(layer),
                    hidden_states,
                    extended_attention_mask,
                )
            else:
                hidden_states = layer(hidden_states, extended_attention_mask)
            
        # MLM loss calculation
        if masked_lm_labels is not None:
            prediction_scores = self.mlm_head(hidden_states)
            loss_fct = nn.CrossEntropyLoss(ignore_index=-1)
            masked_lm_loss = loss_fct(prediction_scores.view(-1, self.config.vocab_size), 
                                    masked_lm_labels.view(-1))
            return masked_lm_loss
            
        return hidden_states
    
    def enable_gradient_checkpointing(self):
        self.gradient_checkpointing = True

def pad_sequence(tokens, max_len, pad_token):
    """Pad or truncate a sequence to max_len."""
    if len(tokens) > max_len:
        return tokens[:max_len]
    return tokens + [pad_token] * (max_len - len(tokens))

def prepare_batch(texts, word2idx, config):
    """Prepare a batch of texts for BERT training."""
    # Convert texts to token ids and pad
    batch_input_ids = []
    for text in texts:
        tokens = text.split()
        token_ids = [word2idx.get(word, word2idx['[UNK]']) for word in tokens]
        # Add [CLS] at start and [SEP] at end
        token_ids = [word2idx['[CLS]']] + token_ids + [word2idx['[SEP]']]
        # Pad sequence
        padded_ids = pad_sequence(token_ids, config.max_len, word2idx['[PAD]'])
        batch_input_ids.append(padded_ids)
    
    # Convert to tensor
    input_ids = torch.tensor(batch_input_ids).to(device)
    attention_mask = (input_ids != word2idx['[PAD]']).float()
    
    return input_ids, attention_mask

def load_and_preprocess_data():
    logger.info("Loading BookCorpus dataset...")
    # Load only 100k samples as specified
    dataset = load_dataset('bookcorpus', split='train[:150000]')
    
    logger.info("Preprocessing text data...")
    texts = dataset['text']
    texts = [text.lower() for text in texts]
    texts = [re.sub(r'[^\w\s]', '', text) for text in texts]
    
    # Create vocabulary
    logger.info("Creating vocabulary...")
    word_set = set()
    for text in texts:
        words = text.split()
        word_set.update(words)
    
    # Add special tokens
    vocab = ['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]'] + list(word_set)
    word2idx = {word: idx for idx, word in enumerate(vocab)}
    
    return texts, word2idx, vocab

def save_model_and_config(model, config, epoch, loss, save_dir='model_checkpoints'):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    model_path = os.path.join(save_dir, f'bert_epoch_{epoch}_{timestamp}.pt')
    config_path = os.path.join(save_dir, f'config_{timestamp}.json')
    
    # Save model
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'loss': loss,
    }, model_path)
    
    # Save config
    config_dict = {k: v for k, v in vars(config).items() if not k.startswith('__')}
    with open(config_path, 'w') as f:
        json.dump(config_dict, f, indent=4)
    
    logger.info(f"Model saved to {model_path}")
    logger.info(f"Config saved to {config_path}")

def main():
    logger.info("Starting BERT training from scratch")
    
    # Enable mixed precision training with proper initialization
    scaler = torch.amp.GradScaler(enabled=True)
    
    # Load and preprocess data
    texts, word2idx, vocab = load_and_preprocess_data()
    
    # Initialize config
    config = BertConfig()
    config.vocab_size = len(vocab)
    logger.info(f"Vocabulary size: {config.vocab_size}")
    
    # Initialize model with gradient checkpointing
    model = BertModel(config).to(device)
    model.enable_gradient_checkpointing()
    logger.info("Model initialized with gradient checkpointing")
    
    # Initialize optimizer with weight decay and proper learning rate
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {
            'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            'weight_decay': 0.01
        },
        {
            'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            'weight_decay': 0.0
        }
    ]
    optimizer = optim.AdamW(optimizer_grouped_parameters, lr=config.learning_rate, eps=1e-8)
    
    # Add learning rate scheduler with warmup
    num_training_steps = len(texts) // (config.batch_size * config.gradient_accumulation_steps) * 10
    num_warmup_steps = num_training_steps // 10
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps,
        num_cycles=0.5,  
    )
    
    # Training loop
    logger.info("Starting training...")
    model.train()
    
    try:
        for epoch in range(15):
            total_loss = 0
            valid_loss_count = 0
            optimizer.zero_grad()  # Reset gradients at start of epoch
            
            progress_bar = tqdm(range(0, len(texts), config.batch_size), 
                              desc=f"Epoch {epoch+1}")
            
            for step, batch_start in enumerate(progress_bar):
                batch_texts = texts[batch_start:batch_start + config.batch_size]
                
                # Prepare batch data with padding
                input_ids, attention_mask = prepare_batch(batch_texts, word2idx, config)
                
                # Create masked tokens
                masked_labels = input_ids.clone()
                special_tokens = {word2idx['[PAD]'], word2idx['[CLS]'], word2idx['[SEP]']}
                mask_candidates = torch.ones_like(input_ids, device=device).bool()
                for special_token in special_tokens:
                    mask_candidates &= (input_ids != special_token)
                
                # Apply masking with 15% probability to valid tokens
                mask_prob = torch.full(input_ids.shape, 0.15, device=device)
                mask = (torch.bernoulli(mask_prob).bool() & mask_candidates)
                
                masked_labels[~mask] = -1  # Only compute loss on masked tokens
                input_ids[mask] = word2idx['[MASK]']
                
                try:
                    # Mixed precision forward pass
                    with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                        loss = model(
                            input_ids,
                            attention_mask=attention_mask,
                            masked_lm_labels=masked_labels
                        )
                        
                        # Scale loss for gradient accumulation
                        loss = loss / config.gradient_accumulation_steps
                        
                        # Check if loss is valid
                        if not torch.isfinite(loss):
                            logger.warning(f"Non-finite loss detected: {loss.item()}")
                            continue
                    
                    # Backward pass with gradient scaling
                    scaler.scale(loss).backward()
                    
                    # Gradient accumulation
                    if (step + 1) % config.gradient_accumulation_steps == 0:
                        # Clip gradients
                        scaler.unscale_(optimizer)
                        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                        
                        # Optimizer step
                        scaler.step(optimizer)
                        scaler.update()
                        scheduler.step()
                        optimizer.zero_grad()
                    
                    # Update loss statistics
                    loss_value = loss.item() * config.gradient_accumulation_steps
                    if np.isfinite(loss_value):
                        total_loss += loss_value
                        valid_loss_count += 1
                    
                    progress_bar.set_postfix({
                        'loss': loss_value,
                        'lr': scheduler.get_last_lr()[0]
                    })
                    
                except RuntimeError as e:
                    if "out of memory" in str(e):
                        logger.warning(f"Out of memory in batch. Skipping batch and clearing cache.")
                        if hasattr(torch.cuda, 'empty_cache'):
                            torch.cuda.empty_cache()
                        optimizer.zero_grad()
                        continue
                    raise e
                
                # Clear cache periodically
                if step % 100 == 0 and hasattr(torch.cuda, 'empty_cache'):
                    torch.cuda.empty_cache()
            
            # Calculate average loss only from valid losses
            avg_loss = total_loss / valid_loss_count if valid_loss_count > 0 else float('nan')
            logger.info(f"Epoch {epoch+1} completed. Average loss: {avg_loss:.4f} (from {valid_loss_count} valid batches)")
            
            # Save model checkpoint
            save_model_and_config(model, config, epoch+1, avg_loss)
        
        logger.info("Training completed!")
        
    except RuntimeError as e:
        if "out of memory" in str(e):
            logger.error(f"GPU out of memory error: {e}")
            logger.info("Try reducing batch_size or model size further if this error persists")
            if hasattr(torch.cuda, 'empty_cache'):
                torch.cuda.empty_cache()
        raise e

if __name__ == "__main__":
    main()


2025-02-17 16:42:15,492 - INFO - Selected GPU 1 with 11004.50MB free memory
2025-02-17 16:42:15,494 - INFO - Using device: cuda:1
2025-02-17 16:42:15,502 - INFO - Starting BERT training from scratch
2025-02-17 16:42:15,503 - INFO - Loading BookCorpus dataset...
2025-02-17 16:42:19,577 - INFO - Preprocessing text data...
2025-02-17 16:42:20,363 - INFO - Creating vocabulary...
2025-02-17 16:42:20,579 - INFO - Vocabulary size: 27092
2025-02-17 16:42:20,990 - INFO - Model initialized with gradient checkpointing
2025-02-17 16:42:20,992 - INFO - Starting training...


Epoch 1:   0%|          | 0/2344 [00:00<?, ?it/s]

/home/jupyter-st125462/.local/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
2025-02-17 16:45:13,474 - INFO - Epoch 1 completed. Average loss: 7.9539 (from 2344 valid batches)
2025-02-17 16:45:13,590 - INFO - Model saved to model_checkpoints/bert_epoch_1_20250217_164513.pt
2025-02-17 16:45:13,591 - INFO - Config saved to model_checkpoints/config_20250217_164513.json


Epoch 2:   0%|          | 0/2344 [00:00<?, ?it/s]

2025-02-17 16:48:06,279 - INFO - Epoch 2 completed. Average loss: 6.3747 (from 2344 valid batches)
2025-02-17 16:48:06,383 - INFO - Model saved to model_checkpoints/bert_epoch_2_20250217_164806.pt
2025-02-17 16:48:06,383 - INFO - Config saved to model_checkpoints/config_20250217_164806.json


...

2025-02-17 17:19:32,317 - INFO - Epoch 13 completed. Average loss: 5.1567 (from 2344 valid batches)
2025-02-17 17:19:32,422 - INFO - Model saved to model_checkpoints/bert_epoch_13_20250217_171932.pt
2025-02-17 17:19:32,423 - INFO - Config saved to model_checkpoints/config_20250217_171932.json


Epoch 14:   0%|          | 0/2344 [00:00<?, ?it/s]

2025-02-17 17:22:23,889 - INFO - Epoch 14 completed. Average loss: 5.1155 (from 2344 valid batches)
2025-02-17 17:22:23,993 - INFO - Model saved to model_checkpoints/bert_epoch_14_20250217_172223.pt
2025-02-17 17:22:23,994 - INFO - Config saved to model_checkpoints/config_20250217_172223.json


Epoch 15:   0%|          | 0/2344 [00:00<?, ?it/s]

2025-02-17 17:25:15,268 - INFO - Epoch 15 completed. Average loss: 5.0737 (from 2344 valid batches)
2025-02-17 17:25:15,367 - INFO - Model saved to model_checkpoints/bert_epoch_15_20250217_172515.pt
2025-02-17 17:25:15,368 - INFO - Config saved to model_checkpoints/config_20250217_172515.json
2025-02-17 17:25:15,369 - INFO - Training completed!


### Findings

[Training Logs can be found in the logs directory in the a4_do_you_agree/bert_training.log]

## Hardware and Initialization
- Training used GPU 1 with approximately 11GB of free memory
- Device: CUDA-enabled GPU (cuda:1)

## Dataset and Model Setup
- Successfully loaded BookCorpus dataset
- Vocabulary size: 27,092 tokens
- Model initialized with gradient checkpointing enabled

## Training Progress
- Total epochs completed: 15
- Valid batches per epoch: 2,344
- Training duration: Approximately 43 minutes (16:42 to 17:25)

## Loss Progression
Key loss values across epochs:
- Epoch 1: 7.9539
- Epoch 5: 5.5129
- Epoch 10: 5.1990
- Epoch 15: 5.0737 (final)

## Model Performance Analysis
- Strong initial improvement: Loss dropped significantly from 7.95 to 6.37 between epochs 1-2
- Steady convergence: Loss continued to decrease gradually
- Final improvement: ~36% reduction in loss from start (7.95) to finish (5.07)

## Technical Notes
- Model checkpoints and configurations were saved after each epoch
- Some widget display errors were logged but didn't affect training
- Warning about torch.utils.checkpoint parameter usage was recorded

The training completed successfully with a clear trend of decreasing loss, indicating effective model learning.

# Task 2: Task 2. Sentence Embedding with Sentence BERT✅

Implement trained BERT from task 1 with siamese network structures to derive semantically meaningful sentence embeddings that can be compared
using cosine-similarity. (3 points)✅
1) Use the SNLI 4 OR MNLI 5 datasets from Hugging Face, or any dataset related to classification tasks.✅
2) Reproduce training the Sentence-BERT as described in the paper 6.✅
3) Focus on the Classification Objective Function: (SoftmaxLoss)

o = softmax 1W T · (u, v, |u − v|)2✅

HINT : You can take a look how to implement Softmax loss in the file 04 - Huggingface/Appendix - Sentence Embedding/S-BERT.ipynb.

In [7]:
import os
import logging
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_dataset, concatenate_datasets
from transformers import BertTokenizer
from tqdm.auto import tqdm
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
import json
from datetime import datetime

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('sbert_training.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

class SentenceBERT(nn.Module):
    def __init__(self, bert_model=None, hidden_size=256, config=None):
        super().__init__()
        if bert_model is None:
            from bert_scratch import BertModel, BertConfig
            if config is None:
                config = BertConfig()
                config.hidden_size = hidden_size
            self.bert = BertModel(config)
        else:
            self.bert = bert_model
            
        self.fc = nn.Linear(hidden_size * 3, 3)  # 3 classes: entailment, contradiction, neutral
        
    def mean_pooling(self, token_embeddings, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
    def encode(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        if isinstance(outputs, tuple):
            outputs = outputs[0]  # Get hidden states if tuple is returned
        embeddings = self.mean_pooling(outputs, attention_mask)
        return embeddings
    
    def forward(self, premise_input_ids, premise_attention_mask, 
               hypothesis_input_ids, hypothesis_attention_mask):
        # Get embeddings for premise and hypothesis
        premise_embedding = self.encode(premise_input_ids, premise_attention_mask)
        hypothesis_embedding = self.encode(hypothesis_input_ids, hypothesis_attention_mask)
        
        # Calculate cosine similarity
        cos_sim = F.cosine_similarity(premise_embedding, hypothesis_embedding)
        self.last_cos_sim = cos_sim  # Store for later use
        
        # Concatenate embeddings
        combined = torch.cat([
            premise_embedding,
            hypothesis_embedding,
            torch.abs(premise_embedding - hypothesis_embedding)
        ], dim=1)
        
        # Pass through classifier
        logits = self.fc(combined)
        
        return logits, cos_sim  # Return both logits and cosine similarity

def load_datasets(num_samples=None):
    logger.info("Loading SNLI and MNLI datasets...")
    
    # Load SNLI dataset
    snli_dataset = load_dataset("snli")
    
    # Load MNLI dataset
    mnli_dataset = load_dataset("multi_nli")
    
    # Rename MNLI labels to match SNLI
    def rename_labels(example):
        label_map = {0: 0, 1: 1, 2: 2}  # entailment: 0, contradiction: 1, neutral: 2
        example['label'] = label_map[example['label']]
        return example
    
    # Process MNLI dataset to match SNLI format
    mnli_dataset = mnli_dataset.map(rename_labels)
    
    # Combine datasets
    train_dataset = concatenate_datasets([
        snli_dataset['train'],
        mnli_dataset['train']
    ])
    
    val_dataset = concatenate_datasets([
        snli_dataset['validation'],
        mnli_dataset['validation_matched']
    ])
    
    # Subsample if specified
    if num_samples is not None:
        train_dataset = train_dataset.shuffle(seed=42).select(range(num_samples))
        val_dataset = val_dataset.shuffle(seed=42).select(range(num_samples))
    
    logger.info(f"Loaded {len(train_dataset)} training samples and {len(val_dataset)} validation samples")
    
    return {
        'train': train_dataset,
        'validation': val_dataset
    }

def preprocess_data(datasets, tokenizer, max_length=128):
    logger.info("Preprocessing datasets...")
    
    def preprocess_function(examples):
        # Tokenize premises
        premise_encodings = tokenizer(
            examples['premise'],
            padding='max_length',
            truncation=True,
            max_length=max_length
        )
        
        # Tokenize hypotheses
        hypothesis_encodings = tokenizer(
            examples['hypothesis'],
            padding='max_length',
            truncation=True,
            max_length=max_length
        )
        
        return {
            'premise_input_ids': premise_encodings['input_ids'],
            'premise_attention_mask': premise_encodings['attention_mask'],
            'hypothesis_input_ids': hypothesis_encodings['input_ids'],
            'hypothesis_attention_mask': hypothesis_encodings['attention_mask'],
            'labels': examples['label']
        }
    
    # Process each split
    processed_datasets = {}
    for split, dataset in datasets.items():
        processed_datasets[split] = dataset.map(
            preprocess_function,
            batched=True,
            remove_columns=dataset.column_names
        )
        processed_datasets[split].set_format('torch')
    
    return processed_datasets

def evaluate_model(model, dataloader):
    model.eval()
    all_predictions = []
    all_labels = []
    all_cosine_sims = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            # Get predictions
            outputs, cos_sim = model(
                batch['premise_input_ids'].to(device),
                batch['premise_attention_mask'].to(device),
                batch['hypothesis_input_ids'].to(device),
                batch['hypothesis_attention_mask'].to(device)
            )
            predictions = torch.argmax(outputs, dim=1)
            
            # Collect predictions and labels
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(batch['labels'].numpy())
            all_cosine_sims.extend(cos_sim.cpu().numpy())
    
    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_predictions)
    report = classification_report(all_labels, all_predictions)
    
    # Calculate average cosine similarity for each class
    cosine_sims = np.array(all_cosine_sims)
    labels = np.array(all_labels)
    
    logger.info("\nCosine Similarity Analysis:")
    for label in [0, 1, 2]:  # entailment, contradiction, neutral
        mask = labels == label
        if mask.any():
            label_name = ['entailment', 'contradiction', 'neutral'][label]
            sims = cosine_sims[mask]
            logger.info(f"{label_name.capitalize()}: Mean={sims.mean():.4f}, Std={sims.std():.4f}")
    
    return accuracy, report

def save_model(model, tokenizer, config, metrics, output_dir='sbert_model'):
    """Save the model, tokenizer, configuration and metrics."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Save model state
    model_path = os.path.join(output_dir, 'model.pt')
    torch.save(model.state_dict(), model_path)
    logger.info(f"Model saved to {model_path}")
    
    # Save tokenizer
    tokenizer_path = os.path.join(output_dir, 'tokenizer')
    tokenizer.save_pretrained(tokenizer_path)
    logger.info(f"Tokenizer saved to {tokenizer_path}")
    
    # Save metrics
    metrics_path = os.path.join(output_dir, 'metrics.json')
    with open(metrics_path, 'w') as f:
        json.dump(metrics, f, indent=4)
    logger.info(f"Metrics saved to {metrics_path}")
    
    # Save configuration
    config_path = os.path.join(output_dir, 'config.json')
    with open(config_path, 'w') as f:
        json.dump(config.__dict__, f, indent=4)
    logger.info(f"Configuration saved to {config_path}")

def main():
    # Load datasets with smaller batch size
    datasets = load_datasets(num_samples=800)
    logger.info(f"Loaded {len(datasets['train'])} training samples and {len(datasets['validation'])} validation samples")
    
    # Initialize tokenizer and get vocab size
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    vocab_size = len(tokenizer.vocab)
    logger.info(f"Tokenizer vocabulary size: {vocab_size}")
    
    # Load pre-trained BERT from our custom implementation
    from bert_scratch import BertModel, BertConfig
    config = BertConfig()
    config.vocab_size = vocab_size  # Set vocab size to match tokenizer
    config.batch_size = 2  # Extremely small batch size
    config.hidden_size = 64  # Minimal hidden size
    config.num_hidden_layers = 2  # Minimal number of layers
    config.num_attention_heads = 4  # Reduced attention heads
    config.gradient_accumulation_steps = 16  # Heavy gradient accumulation
    config.intermediate_size = 256  # Minimal intermediate size
    config.max_len = 32  # Minimal sequence length
    config.attention_probs_dropout_prob = 0.1
    config.hidden_dropout_prob = 0.1
    
    bert_model = BertModel(config)
    
    # Try to load pre-trained weights if available
    try:
        checkpoints = [f for f in os.listdir('model_checkpoints') if f.startswith('bert_epoch_')]
        if checkpoints:
            latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('_')[2]))
            checkpoint_path = os.path.join('model_checkpoints', latest_checkpoint)
            
            # Load checkpoint with proper handling
            checkpoint = torch.load(checkpoint_path, map_location='cpu')  # Load to CPU first
            if 'model_state_dict' in checkpoint:
                state_dict = checkpoint['model_state_dict']
            else:
                state_dict = checkpoint
            
            # Filter out mismatched keys
            model_dict = bert_model.state_dict()
            state_dict = {k: v for k, v in state_dict.items() 
                         if k in model_dict and v.shape == model_dict[k].shape}
            
            # Load filtered state dict
            bert_model.load_state_dict(state_dict, strict=False)
            logger.info(f"Loaded compatible weights from {checkpoint_path}")
        else:
            logger.warning("No pre-trained BERT checkpoints found. Starting with random initialization.")
    except Exception as e:
        logger.warning(f"Failed to load pre-trained BERT weights: {e}")
    
    # Initialize Sentence-BERT
    model = SentenceBERT(bert_model, hidden_size=config.hidden_size, config=config)
    
    # Enable gradient checkpointing for memory efficiency
    if hasattr(model.bert, 'enable_gradient_checkpointing'):
        model.bert.enable_gradient_checkpointing()
        logger.info("Enabled gradient checkpointing")
    
    # Move model to GPU after all initialization
    model = model.to(device)
    
    # Clear GPU memory before starting
    torch.cuda.empty_cache()
    
    # Preprocess datasets with reduced sequence length
    tokenized_datasets = preprocess_data(datasets, tokenizer, max_length=config.max_len)
    
    # Create dataloaders with minimal batch size
    train_dataloader = DataLoader(
        tokenized_datasets['train'],
        batch_size=config.batch_size,
        shuffle=True,
        pin_memory=True,
        num_workers=0,
        persistent_workers=False
    )
    
    val_dataloader = DataLoader(
        tokenized_datasets['validation'],
        batch_size=config.batch_size,
        pin_memory=True,
        num_workers=0,
        persistent_workers=False
    )
    
    # Initialize optimizer with weight decay
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {
            'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            'weight_decay': config.weight_decay
        },
        {
            'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            'weight_decay': 0.0
        }
    ]
    
    optimizer = optim.AdamW(
        optimizer_grouped_parameters,
        lr=config.learning_rate,
        eps=config.adam_epsilon
    )
    
    # Training loop
    num_epochs = 10
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
    
    try:
        for epoch in range(num_epochs):
            model.train()
            total_loss = 0
            epoch_cos_sims = []
            
            progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
            for step, batch in enumerate(progress_bar):
                # Zero gradients
                optimizer.zero_grad()
                
                # Forward pass
                outputs, cos_sim = model(
                    batch['premise_input_ids'].to(device),
                    batch['premise_attention_mask'].to(device),
                    batch['hypothesis_input_ids'].to(device),
                    batch['hypothesis_attention_mask'].to(device)
                )
                
                # Calculate loss
                loss = criterion(outputs, batch['labels'].to(device))
                
                # Backward pass
                loss.backward()
                
                # Clip gradients
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                
                # Update parameters
                optimizer.step()
                scheduler.step()
                
                # Update metrics
                total_loss += loss.item()
                epoch_cos_sims.extend(cos_sim.detach().cpu().numpy())
                
                # Update progress bar
                progress_bar.set_postfix({'loss': total_loss / (step + 1)})
                
                # Cleanup
                del outputs, loss, batch
                torch.cuda.empty_cache()
            
            # Calculate epoch metrics
            avg_loss = total_loss / len(train_dataloader)
            epoch_cos_sims = np.array(epoch_cos_sims)
            
            # Print epoch summary
            logger.info(f"Epoch {epoch+1} completed. Average loss: {avg_loss:.4f}")
            logger.info(f"Epoch {epoch+1} Cosine Similarities - Mean: {epoch_cos_sims.mean():.4f}, Std: {epoch_cos_sims.std():.4f}")
            
            # Evaluate
            accuracy, report = evaluate_model(model, val_dataloader)
            logger.info(f"Validation metrics: ({accuracy}, '{report}')")
            
            # Save model
            metrics_dict = {
                'accuracy': float(accuracy),
                'loss': float(avg_loss),
                'classification_report': report,
                'cosine_similarity_mean': float(epoch_cos_sims.mean()),
                'cosine_similarity_std': float(epoch_cos_sims.std())
            }
            save_model(model, tokenizer, config, metrics_dict, output_dir='sbert_model')
            
    except KeyboardInterrupt:
        logger.info("Training interrupted by user")
    except Exception as e:
        logger.error(f"Error during training: {e}")
        raise
    finally:
        logger.info("Training completed!")

if __name__ == "__main__":
    main()


2025-02-17 17:30:52,384 - INFO - Using device: cuda
2025-02-17 17:30:52,391 - INFO - Loading SNLI and MNLI datasets...
2025-02-17 17:31:08,866 - INFO - Loaded 800 training samples and 800 validation samples
2025-02-17 17:31:08,868 - INFO - Loaded 800 training samples and 800 validation samples
2025-02-17 17:31:09,210 - INFO - Tokenizer vocabulary size: 30522
/tmp/ipykernel_1859902/3502560219.py:259: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly 

Epoch 1:   0%|          | 0/400 [00:00<?, ?it/s]

2025-02-17 17:31:31,000 - INFO - Epoch 1 completed. Average loss: 1.1087
2025-02-17 17:31:31,002 - INFO - Epoch 1 Cosine Similarities - Mean: 0.9317, Std: 0.0296


Evaluating:   0%|          | 0/400 [00:00<?, ?it/s]

/home/jupyter-st125462/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jupyter-st125462/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jupyter-st125462/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

...

/home/jupyter-st125462/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jupyter-st125462/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jupyter-st125462/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

Epoch 10:   0%|          | 0/400 [00:00<?, ?it/s]

/home/jupyter-st125462/.local/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
2025-02-17 17:35:12,721 - INFO - Epoch 10 completed. Average loss: 1.0340
2025-02-17 17:35:12,722 - INFO - Epoch 10 Cosine Similarities - Mean: 0.7670, Std: 0.1534


Evaluating:   0%|          | 0/400 [00:00<?, ?it/s]

/home/jupyter-st125462/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jupyter-st125462/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jupyter-st125462/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

### Findings

## BERT Pre-training Results
- Training completed 15 epochs
- Initial vocabulary size: 27,092 tokens
- Training device: GPU 1 with 11GB free memory
- Loss progression:
  - Starting loss: 7.9539 (Epoch 1)
  - Final loss: 5.0737 (Epoch 15)
  - Overall loss reduction: ~36%

## SBERT Fine-tuning Results
- Dataset: Combined SNLI and MNLI (800 training + 800 validation samples)
- Model configuration:
  - Vocabulary size: 30,522
  - Hidden size: 64
  - Layers: 2
  - Attention heads: 4

### Final Performance Metrics (After 10 epochs)

| Metric | Value |
|--------|--------|
| Accuracy | 40.25% |
| Average Loss | 1.0340 |
| Cosine Similarity Mean | 0.7670 |
| Cosine Similarity Std | 0.1534 |

### Class-wise Performance
| Class | Precision | Recall | F1-Score |
|-------|-----------|--------|-----------|
| Entailment | 0.43 | 0.38 | 0.41 |
| Contradiction | 0.41 | 0.61 | 0.49 |
| Neutral | 0.33 | 0.21 | 0.25 |

### Training Progression
- Started with high cosine similarities (~0.93)
- Gradually decreased to more discriminative values (~0.77)
- Model showed steady improvement in classification performance
- Final weighted average metrics:
  - Precision: 0.39
  - Recall: 0.40
  - F1-score: 0.38

Both training sessions completed successfully with proper model and checkpoint saving at each epoch.

# Task 3. Evaluation and Analysis (1 points) ✅

1) Provide the performance metrics based on the SNLI or MNLI datasets for the Natural Language
Inference (NLI) task. ✅

2) Discuss any limitations or challenges encountered during the implementation and propose potential
improvements or modifications. ✅


NOTE: Make sure to provide proper documentation, including details of the datasets used, hyperparameters, and any modifications made to the original models

----------------

## 1. Performance Metrics

### Table 1. Performance Table (SNLI + MNLI Combined Dataset)
| Metric | Value |
|--------|--------|
| Overall Accuracy | 40.25% |
| Average Loss | 1.0340 |
| Macro Avg F1-score | 0.29 |
| Weighted Avg F1-score | 0.38 |

### Class-wise Performance
| Class | Precision | Recall | F1-Score |
|-------|-----------|--------|-----------|
| Entailment | 0.43 | 0.38 | 0.41 |
| Contradiction | 0.41 | 0.61 | 0.49 |
| Neutral | 0.33 | 0.21 | 0.25 |

### Cosine Similarity Analysis (Final Epoch)
| Class | Mean | Std |
|-------|------|-----|
| Entailment | 0.7780 | 0.1795 |
| Contradiction | 0.8076 | 0.1534 |
| Neutral | 0.7976 | 0.1491 |

## 2. Implementation Details

### Dataset Information
- Combined SNLI and MNLI datasets
- Training samples: 800
- Validation samples: 800
- Tokenizer vocabulary size: 30,522

### Hyperparameters
- Hidden size: 64
- Number of layers: 2
- Attention heads: 4
- Batch size: 2
- Sequence length: 32
- Learning rate: Not explicitly stated in logs
- Training epochs: 10
- Gradient accumulation steps: 16

## 3. Limitations and Challenges

1. **Resource Constraints:**
   - Had to use minimal model architecture due to memory limitations
   - Required gradient checkpointing for memory efficiency
   - Small batch size (2) needed to fit in memory

2. **Performance Limitations:**
   - Relatively low accuracy (40.25%)
   - Poor performance on neutral class (F1: 0.25)
   - High cosine similarities between different classes

3. **Dataset Challenges:**
   - Small training set (800 samples) may not be representative
   - Imbalanced class distribution
   - Some invalid labels present (-1 class with 8 samples)

## 4. Proposed Improvements

1. **Model Architecture:**
   - Increase model capacity (more layers, wider hidden dimensions)
   - Implement attention mechanisms specific to NLI tasks
   - Add residual connections for better gradient flow

2. **Training Strategy:**
   - Use larger batch sizes with gradient accumulation
   - Implement curriculum learning
   - Add contrastive learning objectives

3. **Data Processing:**
   - Use larger training dataset
   - Balance class distribution
   - Better handling of invalid labels
   - Implement data augmentation techniques

4. **Optimization:**
   - Try different learning rate schedules
   - Implement early stopping
   - Use mixed-precision training
   - Add regularization techniques

The implementation shows proof of concept but would benefit from these improvements for production use.

# Task 4. Text similarity - Web Application Development ✅

Develop a simple web application that demonstrates the capabilities of your text-embedding model. (1 points) ✅
1) Develop a simple website with two input boxes for search queries.✅
2) Utilize a custom-trained sentence transformer model to predict Natural Language Inference (NLI)✅

Task (entailment, neutral and contradiction).
For example:
• Premise: A man is playing a guitar on stage.
• Hypothesis: The man is performing music.
• Label: Entailment

The app is available on GitHub Lionks mentioned in the beginning of the notebook!

Below is the inference script for the same:

In [3]:
import torch
import json
from transformers import BertTokenizer
from sentence_bert import SentenceBERT
from bert_scratch import BertConfig
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class NLIPredictor:
    def __init__(self, model_dir='sbert_model'):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        logger.info(f"Using device: {self.device}")
        
        # Initialize tokenizer first to get vocab size
        self.tokenizer = BertTokenizer.from_pretrained(f"{model_dir}/tokenizer")
        logger.info(f"Loaded tokenizer with vocabulary size: {len(self.tokenizer.vocab)}")
        
        # Initialize BERT config with SBERT values
        from bert_scratch import BertConfig
        config = BertConfig()
        
        # Set model architecture parameters
        config.vocab_size = 30522  # Fixed vocab size from SBERT config
        config.hidden_size = 64
        config.num_hidden_layers = 2
        config.num_attention_heads = 4
        config.intermediate_size = 256
        config.max_position_embeddings = 128
        config.max_len = 32
        config.type_vocab_size = 2
        
        # Set dropout and normalization parameters
        config.hidden_dropout_prob = 0.1
        config.attention_probs_dropout_prob = 0.1
        config.layer_norm_eps = 1e-12
        
        # Set special token IDs
        config.pad_token_id = 0
        config.mask_token_id = 3
        config.cls_token_id = 1
        config.sep_token_id = 2
        
        # Training parameters (required by BertConfig)
        config.learning_rate = 1e-4
        config.batch_size = 2
        config.gradient_accumulation_steps = 16
        config.weight_decay = 0.01
        config.adam_epsilon = 1e-8
        config.warmup_ratio = 0.1
        
        # Initialize SentenceBERT with the config
        from sentence_bert import SentenceBERT
        self.model = SentenceBERT(None, hidden_size=config.hidden_size, config=config)
        
        # Load model weights
        try:
            model_path = f"{model_dir}/model.pt"
            state_dict = torch.load(model_path, map_location=self.device)
            self.model.load_state_dict(state_dict)
            logger.info(f"Loaded model weights from {model_path}")
        except Exception as e:
            logger.error(f"Error loading model weights: {e}")
            raise
        
        self.model.to(self.device)
        self.model.eval()
        
        # Label mapping
        self.id2label = {0: 'entailment', 1: 'contradiction', 2: 'neutral'}
        
        logger.info("Model initialized successfully")
    
    def predict(self, premise, hypothesis):
        # Clear GPU cache before prediction
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        # Tokenize inputs
        inputs = self.tokenizer(
            [premise, hypothesis],
            padding=True,
            truncation=True,
            max_length=32,  # Use fixed max_length from config
            return_tensors='pt'
        )
        
        # Split inputs for premise and hypothesis
        premise_input_ids = inputs['input_ids'][0].unsqueeze(0)
        premise_attention_mask = inputs['attention_mask'][0].unsqueeze(0)
        hypothesis_input_ids = inputs['input_ids'][1].unsqueeze(0)
        hypothesis_attention_mask = inputs['attention_mask'][1].unsqueeze(0)
        
        # Move to device
        premise_input_ids = premise_input_ids.to(self.device)
        premise_attention_mask = premise_attention_mask.to(self.device)
        hypothesis_input_ids = hypothesis_input_ids.to(self.device)
        hypothesis_attention_mask = hypothesis_attention_mask.to(self.device)
        
        # Get prediction
        with torch.no_grad():
            with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                outputs = self.model(
                    premise_input_ids,
                    premise_attention_mask,
                    hypothesis_input_ids,
                    hypothesis_attention_mask
                )
                prediction = torch.argmax(outputs, dim=1).item()
                probabilities = torch.nn.functional.softmax(outputs, dim=1)[0]
        
        result = {
            'label': self.id2label[prediction],
            'probabilities': {
                self.id2label[i]: float(prob.item())
                for i, prob in enumerate(probabilities)
            }
        }
        
        # Clear memory
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        return result

def main():
    # Initialize predictor
    predictor = NLIPredictor()
    
    # Example usage
    examples = [
        {
            'premise': 'A man is playing a guitar on stage.',
            'hypothesis': 'The man is performing music.',
            'expected': 'entailment'
        },
        {
            'premise': 'The cat is sleeping on the couch.',
            'hypothesis': 'The dog is running in the park.',
            'expected': 'contradiction'
        },
        {
            'premise': 'A woman is reading a book.',
            'hypothesis': 'She is wearing glasses.',
            'expected': 'neutral'
        },
        {
        'premise': 'Children are playing soccer in the park.',
        'hypothesis': 'Kids are engaged in outdoor sports.',
        'expected': 'entailment'
    },
    {
        'premise': 'The restaurant is packed with customers.',
        'hypothesis': 'The restaurant is closed today.',
        'expected': 'contradiction'
    },
    {
        'premise': 'A student is writing notes in class.',
        'hypothesis': 'The student understands the material.',
        'expected': 'neutral'
    },
    {
        'premise': 'The chef is preparing pasta in the kitchen.',
        'hypothesis': 'Someone is cooking food.',
        'expected': 'entailment'
    },
    {
        'premise': 'The sky is clear and blue today.',
        'hypothesis': 'It is raining heavily.',
        'expected': 'contradiction'
    },
    {
        'premise': 'A person bought a new laptop.',
        'hypothesis': 'They got it from Amazon.',
        'expected': 'neutral'
    },
    {
        'premise': 'The train arrived 30 minutes late.',
        'hypothesis': 'The train was delayed.',
        'expected': 'entailment'
    },
    {
        'premise': 'The museum is open on weekends.',
        'hypothesis': 'The museum is closed every day.',
        'expected': 'contradiction'
    },
    {
        'premise': 'A woman is walking her dog.',
        'hypothesis': 'The dog is brown in color.',
        'expected': 'neutral'
    },
    {
        'premise': 'The movie theater is showing new releases.',
        'hypothesis': 'Films are being screened.',
        'expected': 'entailment'
    }
    ]
    
    # Test each example
    for i, example in enumerate(examples, 1):
        result = predictor.predict(example['premise'], example['hypothesis'])
        logger.info(f"\nExample {i}:")
        logger.info(f"Premise: {example['premise']}")
        logger.info(f"Hypothesis: {example['hypothesis']}")
        logger.info(f"Expected: {example['expected']}")
        logger.info(f"Predicted: {result['label']}")
        logger.info("Probabilities:")
        for label, prob in result['probabilities'].items():
            logger.info(f"  {label}: {prob:.3f}")

if __name__ == "__main__":
    main()


2025-02-22 08:43:49,069 - INFO - Using device: cuda
2025-02-22 08:43:49,120 - INFO - Selected GPU 0 with 11004.50MB free memory
2025-02-22 08:43:49,121 - INFO - Using device: cuda:0
2025-02-22 08:43:49,123 - INFO - Using device: cuda
2025-02-22 08:43:49,165 - INFO - Loaded tokenizer with vocabulary size: 30522
/tmp/ipykernel_2166839/3675333752.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.a

# Cool and Interesting Insights!

## Confidence Patterns
1. High Confidence Successes
- Most successful entailment predictions had very high confidence (>0.7)
- Example: "chef preparing pasta" → "someone cooking food" (86.8% confidence)
- Example: "movie theater showing releases" → "films being screened" (71.1% confidence)

2. Challenging Cases
- Model struggles most with neutral relationships, often misclassifying them as contradictions
- Borderline cases show very close probability distributions across all three classes
- Example: "train delayed" case had almost equal probabilities (35.6%, 35.0%, 29.4%)

## Pattern Analysis
1. Strong Performance:
- Direct logical entailments (cooking→food preparation)
- Simple activity relationships (playing soccer→outdoor sports)
- Direct contradictions with clear opposing statements

2. Common Mistakes:
- Over-predicting entailment for temporal relationships
- Struggling with neutral cases involving additional details
- Difficulty with implicit contradictions

## Specific Weaknesses
1. Context Understanding:
- Failed on "clear sky" → "heavy rain" (predicted entailment with 77.3% confidence)
- Struggled with context-dependent relationships

2. Subtle Relationships:
- Poor performance on neutral cases requiring world knowledge
- Example: "student writing notes" → "understanding material"
- Example: "laptop purchase" → "Amazon purchase"

The model shows promising performance on straightforward relationships but needs improvement in handling subtle distinctions and world knowledge integration.

# Thank You! 🤗